In [1119]:
import pandas as pd
import numpy as np
from sklearn.cluster import HDBSCAN
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import cdist

## Importazione dati di lavoro
Questa sezione andrà riscritta quando faremo i collegamenti client-server e da/verso il database

### Importiamo i dati
Qui sarà la parte dove importiamo i dati dal database

In [ ]:
df1 = pd.read_csv("dataset2.csv")
df = df1.drop(columns=["name", "bio", "gender", "gender_other", "age_flag_other",
                           "age_radius_other", "distance_flag_other", "distance_km_other",
                           "same_interests"])
df.describe()

,ID,age,longitude,latitude,attractiveness,sincerity,intelligence,funniness,ambition,sports,...,hiking_other,gaming_other,clubbing_other,reading_other,tv_other,theater_other,movies_other,music_other,shopping_other,yoga_other
count,234.000000,234.000000,234.000000,234.000000,234.000000,234.000000,234.000000,234.000000,234.000000,234.000000,...,234.000000,234.000000,234.000000,234.000000,234.000000,234.000000,234.000000,234.000000,234.000000,234.000000
mean,4158.508547,26.559829,10.269286,44.846729,7.123932,7.354701,5.893162,6.393162,6.175214,5.572650,...,5.435897,4.029915,5.470085,4.683761,5.273504,6.119658,5.679487,5.803419,5.777778,5.350427
std,2459.824869,3.684827,0.499163,0.355461,1.141127,0.882738,1.377827,0.993172,1.204099,2.041695,...,2.112235,1.634907,2.059512,1.756164,1.983295,1.941914,2.080939,1.694421,1.923923,2.095637
min,11.000000,21.000000,9.256040,44.098900,3.000000,2.000000,1.000000,3.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
25%,1961.500000,24.000000,9.906305,44.572875,7.000000,7.000000,5.000000,6.000000,6.000000,4.000000,...,3.000000,3.000000,3.250000,4.000000,3.000000,5.000000,4.000000,4.000000,4.000000,3.000000
50%,4056.000000,26.000000,10.273850,44.933250,7.000000,8.000000,6.000000,6.000000,6.000000,6.000000,...,5.000000,4.000000,6.000000,5.000000,5.000000,6.000000,6.000000,6.000000,6.000000,6.000000
75%,6170.750000,28.750000,10.739300,45.153800,8.000000,8.000000,7.000000,7.000000,7.000000,7.000000,...,7.000000,5.000000,7.000000,6.000000,7.000000,8.000000,8.000000,7.000000,7.000000,7.000000
max,8593.000000,38.000000,11.014200,45.350000,9.000000,8.000000,8.000000,8.000000,8.000000,9.000000,...,9.000000,7.000000,9.000000,7.000000,8.000000,9.000000,8.000000,8.000000,9.000000,9.000000


### Creazione del punto di ricerca
Per il momento prendiamo un dato a caso dal dataset come nostro candidato

In [1122]:
search = df.sample()
print(search)

       ID  age  longitude  latitude  attractiveness  sincerity  intelligence  \
129  4532   27    10.1956   44.9154               6          8             6   

     funniness  ambition  sports  ...  hiking_other  gaming_other  \
129          7         5       7  ...             5             6   

     clubbing_other  reading_other  tv_other  theater_other  movies_other  \
129               5              1         3              3             6   

     music_other  shopping_other  yoga_other  
129            8               5           6  

[1 rows x 45 columns]


Costruiamo un punto ad-hoc e salviamo i pesi

In [1123]:
search["age"] = search["age_other"]
search["sports"] = search["sports_other"]
search["tv_sports"] = search["tv_sports_other"]
search["exercise"] = search["exercise_other"]
search["dining"] = search["dining_other"]
search["hiking"] = search["hiking_other"]
search["gaming"] = search["gaming_other"]
search["clubbing"] = search["clubbing_other"]
search["reading"] = search["reading_other"]
search["tv"] = search["tv_other"]
search["theater"] = search["theater_other"]
search["movies"] = search["movies_other"]
search["music"] = search["music_other"]
search["shopping"] = search["shopping_other"]
search["yoga"] = search["yoga_other"]
weights = search[['attractiveness_important', 'sincerity_important',
                    'intelligence_important', 'funniness_important',
                    'ambition_important']].to_numpy(dtype=np.uint8)[0]

In [1124]:
search = search.drop(columns=(["age_other"] + list(df.filter(regex='.*other$')) + list(df.filter(regex='.*important$'))))
print(search)
print(search.columns)

       ID  age  longitude  latitude  attractiveness  sincerity  intelligence  \
129  4532   22    10.1956   44.9154               6          8             6   

     funniness  ambition  sports  ...  hiking  gaming  clubbing  reading  tv  \
129          7         5       4  ...       5       6         5        1   3   

     theater  movies  music  shopping  yoga  
129        3       6      8         5     6  

[1 rows x 24 columns]
Index(['ID', 'age', 'longitude', 'latitude', 'attractiveness', 'sincerity',
       'intelligence', 'funniness', 'ambition', 'sports', 'tv_sports',
       'exercise', 'dining', 'art', 'hiking', 'gaming', 'clubbing', 'reading',
       'tv', 'theater', 'movies', 'music', 'shopping', 'yoga'],
      dtype='object')


In [1125]:
search.iloc[:, 4:9] = 0
print(search)
print(weights)

       ID  age  longitude  latitude  attractiveness  sincerity  intelligence  \
129  4532   22    10.1956   44.9154               0          0             0   

     funniness  ambition  sports  ...  hiking  gaming  clubbing  reading  tv  \
129          0         0       4  ...       5       6         5        1   3   

     theater  movies  music  shopping  yoga  
129        3       6      8         5     6  

[1 rows x 24 columns]
[1 3 3 3 1]


Prepariamo il dataset alla ricerca.

Salviamo gli ID utenti per poter recuperare tutte le loro informazioni alla fine
Prepariamo il dataset alla ricerca. 

In [1126]:
df = df[df["ID"] != search.iloc[0, 0]]
IDs = df["ID"].to_list()
IDs.append(search.iloc[0,0])
IDs = np.array(IDs)
df = df.drop(columns=(["age_other", "ID"] + list(df.filter(regex='.*other$')) + list(df.filter(regex='.*important$'))))
print(df.columns)
search = search.drop(columns=["ID"])
print(search.columns)

Index(['age', 'longitude', 'latitude', 'attractiveness', 'sincerity',
       'intelligence', 'funniness', 'ambition', 'sports', 'tv_sports',
       'exercise', 'dining', 'art', 'hiking', 'gaming', 'clubbing', 'reading',
       'tv', 'theater', 'movies', 'music', 'shopping', 'yoga'],
      dtype='object')
Index(['age', 'longitude', 'latitude', 'attractiveness', 'sincerity',
       'intelligence', 'funniness', 'ambition', 'sports', 'tv_sports',
       'exercise', 'dining', 'art', 'hiking', 'gaming', 'clubbing', 'reading',
       'tv', 'theater', 'movies', 'music', 'shopping', 'yoga'],
      dtype='object')


Convertiamo tutto in numpy array per un'elaborazione numerica più facile, il punto di ricerca sarà l'ultimo punto

In [1127]:
search = search.to_numpy()[0]
X = df.to_numpy()
X = np.vstack((X,search))
print("X:",X, X.shape)

X: [[23.      10.1383  44.7255  ...  7.       9.       5.     ]
 [29.      10.5231  44.7531  ...  8.       6.       8.     ]
 [26.       9.97214 44.9533  ...  8.       7.       6.     ]
 ...
 [28.       9.38892 44.9412  ...  8.       8.       7.     ]
 [24.      10.8498  45.2058  ...  7.       8.       7.     ]
 [22.      10.1956  44.9154  ...  8.       5.       6.     ]] (234, 23)


## Clustering

In [1128]:
clustering = HDBSCAN(min_cluster_size=6, n_jobs=-1, store_centers="centroid", allow_single_cluster=True)
clustering.fit(X[:, np.r_[5:8, 9:23]])

# ri-assegnamo i punti rumorosi
centroids = clustering.centroids_
print("Medoids shape:", centroids.shape)
noisy = X[clustering.labels_ == -1]
noisy = noisy[:,np.r_[5:8, 9:23]]
print("Noisy points shape:",noisy.shape)
distmat = cdist(noisy, centroids)
idxs = np.argmin(distmat, axis=1)
clustering.labels_[clustering.labels_ == -1] = idxs

# filtriamo
print("X before:",X.shape)
X = X[clustering.labels_ == clustering.labels_[-1]]
IDs = IDs[clustering.labels_ == clustering.labels_[-1]]
print("X after:",X.shape)
print(X)

Medoids shape: (1, 17)
Noisy points shape: (217, 17)
X before: (234, 23)
X after: (234, 23)
[[23.      10.1383  44.7255  ...  7.       9.       5.     ]
 [29.      10.5231  44.7531  ...  8.       6.       8.     ]
 [26.       9.97214 44.9533  ...  8.       7.       6.     ]
 ...
 [28.       9.38892 44.9412  ...  8.       8.       7.     ]
 [24.      10.8498  45.2058  ...  7.       8.       7.     ]
 [22.      10.1956  44.9154  ...  8.       5.       6.     ]]


## Deformazione delle variabili
Facciamo valere le preferenze

In [1129]:
exps = [(1/5), (1/1.3), 1, 2, 5]
for cnt, i in enumerate(range(4,9)):
    X[:-1,i] = np.int64(np.float_power((11-X[:-1,i]), exps[weights[cnt]-1]))
print(X)

[[23.      10.1383  44.7255  ...  7.       9.       5.     ]
 [29.      10.5231  44.7531  ...  8.       6.       8.     ]
 [26.       9.97214 44.9533  ...  8.       7.       6.     ]
 ...
 [28.       9.38892 44.9412  ...  8.       8.       7.     ]
 [24.      10.8498  45.2058  ...  7.       8.       7.     ]
 [22.      10.1956  44.9154  ...  8.       5.       6.     ]]


## Nearest Neighbor
È arrivato il momento tanto atteso di trovare i punti più simili a quello di ricerca

In [1130]:
nn = NearestNeighbors(n_jobs=-1)
nn.fit(X)

dsts, idxs = nn.kneighbors(X[-1].reshape(1,-1), return_distance=True, n_neighbors=6)
print("Indici:",idxs)
print("Distanze:",dsts)
print(X[idxs])

Indici: [[233  33  31 135  61   5]]
Distanze: [[ 0.         13.75215272 13.7675659  13.76873735 13.83000819 13.84968383]]
[[[22.      10.1956  44.9154   0.       0.       0.       0.
    0.       4.       6.       2.       4.       4.       5.
    6.       5.       1.       3.       3.       6.       8.
    5.       6.     ]
  [24.      10.2275  45.2628   9.       1.       3.       3.
    3.       1.       5.       2.       7.       6.       6.
    3.       7.       4.       6.       7.       6.       7.
    7.       6.     ]
  [24.      10.8911  45.1647   9.       1.       3.       3.
    3.       1.       5.       2.       7.       6.       6.
    3.       7.       4.       6.       7.       6.       7.
    7.       6.     ]
  [24.       9.94712 44.1968   9.       1.       3.       3.
    3.       1.       5.       2.       7.       6.       6.
    3.       7.       4.       6.       7.       6.       7.
    7.       6.     ]
  [24.      10.6903  45.0716   7.       1.       3.       

## Risultato

In [1098]:
with pd.option_context('display.max_columns', None):
    print("ID punto di ricerca:",IDs[-1])
    print("ID dei punti simili:",IDs[idxs[0, 1:]])
    print("Punto di ricerca:",df1[df1["ID"] == IDs[-1]])
    for i in range(1, idxs.shape[1]):
        print(f"{i}° punto più simile:",df1[df1["ID"] == IDs[idxs[0,i]]])

ID punto di ricerca: 7572
ID dei punti simili: [7398  783 4593 6545  452]
Punto di ricerca:         ID  sports  tvsports  exercise  dining  art  hiking  gaming  clubbing  \
7571  7572     9.0       6.0       6.0     7.0  7.0     8.0     2.0       7.0   

      reading   tv  theater  movies  music  shopping  yoga  sports_partner  \
7571      7.0  2.0      8.0     8.0    8.0       5.0   8.0             9.0   

      tvsports_partner  exercise_partner  dining_partner  art_partner  \
7571               6.0               6.0             7.0          8.0   

      hiking_partner  gaming_partner  clubbing_partner  reading_partner  \
7571             8.0             2.0               8.0              6.0   

      tv_partner  theater_partner  movies_partner  music_partner  \
7571         2.0              8.0             8.0            9.0   

      shopping_partner  yoga_partner  attractive_important  sincere_important  \
7571               6.0           8.0                   1.0              